## 0.0 Pacotes e Bibliotecas

In [1]:
# Manipulação de Dados
import pandas as pd
import numpy  as np

import pickle as pkl

## 1.0 Carregamento dos Dados

In [43]:
df1_fato = pd.read_csv('../BaseDados/DadosTreino.csv', delimiter=',', low_memory=False)
df1_dimensao = pd.read_csv('../BaseDados/DadosLoja.csv', delimiter=',', low_memory=False)
df1 = df1_fato.merge(df1_dimensao, how='left', on='Store')

### 1.1 Seleção dos Dados

Como o nosso objetivo é realizar a previsão dos próximos 6 dias de vendas das lojas, podemos, após um entendimento das variáveis, verificar quais delas fazem sentido. 

1. Ao analisar, temos que duas variávies não fazem sentido para nós. São elas: 'Promo2SinceWeek', 'Promo2SinceYear', 'PromoInterval'.
2. Obtendo apenas os dados dos últimos 7 dias de vendas.

Portanto, vamos remover essas variáveis do nosso dataset e aplicar os filtros, de modo que, iniciemos nosso trabalho.

In [44]:
df1 = df1[df1['Open'] == 1].drop(columns=['Promo2SinceWeek', 'Promo2SinceYear', 'PromoInterval'], axis=1).copy()

## 2.0 Tratamento de Dados

In [53]:
df1 = df2.copy()

In [54]:
# Ajuste da Variável Data
df2['Date'] = pd.to_datetime(df2['Date'], format='%Y-%m-%d')


# Eliminando os dados em que o competition distance é nulo
df2 = df2.drop(index=df2[df2['CompetitionDistance'].isna()].sort_values(by=['Store', 'Date'], ascending=True).index.tolist())

In [55]:
# Obtendo a menor data de vendas em relação a cada loja.
competition_date = df2[['Store', 'Date']].groupby('Store').min().reset_index()

# Vamos extrair o anoa dessas datas
competition_date['MinDate'] =  competition_date['Date'].apply(lambda x: x.year)

# Eliminando colunas desnecessárias
competition_date = competition_date.drop(columns=['Date'], axis=1)

# Aplicação de um left join para preencher os dados nulos
df2 = df2.merge(competition_date, how='left', on='Store')

# Eliminando os Dados Nulos dos Anos
df2['CompetitionOpenSinceYear'] = np.where(df2['CompetitionOpenSinceYear'].isna(), df2['MinDate'], df2['CompetitionOpenSinceYear'])

# Validação
print(f'Quantidade de Dados Nulos: \n{df2.isna().sum()[df2.isna().sum() > 0]}')

Quantidade de Dados Nulos: 
Series([], dtype: int64)


In [56]:
# Obtendo a menor data de vendas em relação a cada loja.
competition_date = df2[['Store', 'Date']].groupby('Store').min().reset_index()

# Vamos extrair o anoa dessas datas
competition_date['MinMonth'] =  competition_date['Date'].apply(lambda x: x.month)

# Eliminando colunas desnecessárias
competition_date = competition_date.drop(columns=['Date'], axis=1)

# Aplicação de um left join para preencher os dados nulos
df2 = df2.merge(competition_date, how='left', on='Store')

# Eliminando os Dados Nulos dos Anos
df2['CompetitionOpenSinceMonth'] = np.where(df2['CompetitionOpenSinceMonth'].isna(), df2['MinDate'], df2['CompetitionOpenSinceMonth'])

# Validação
print(f'Quantidade de Dados Nulos: \n{df2.isna().sum()[df2.isna().sum() > 0]}')

Quantidade de Dados Nulos: 
Series([], dtype: int64)


In [57]:
# Dropando as colunas desnecessárias
df2 = df2.drop(columns=['MinMonth', 'MinDate'], axis=1)

## 3.0 Engenharia de Atributos

O objetivo dessa etapa é conseguir ver a possibilidade de informações que estão 'escondidas' nos dados. Podemos obervar que o nosso dataset é bem limitado, sendo assim, vamos inicialmente construir nossa variável que é as vendas dos próximos 6 dias.

In [69]:
# Cópia dos Dados
df3 = df2.copy()

# Ordenamento dos Dados
df3_sort = df3.sort_values(by=['Store', 'Date'], ascending=True).reset_index(drop=True).copy()

max_data = df3_sort.groupby('Store')['Date'].transform('max')
df3_sort = df3_sort[df3_sort['Date'] >= (max_data - pd.Timedelta(days=7))]

In [70]:
# Construção da variável public holiday
dummies_producao = pd.get_dummies(data=df2_sort['StateHoliday'], prefix='StateHoliday', dtype=np.int64)

if 'StateHoliday_a' in dummies_producao:
    df3_sort['PublicHoliday'] = dummies_producao['StateHoliday_a']
else:
    df3_sort['PublicHoliday'] = 0

In [71]:
# Vendas do último dia
df3_sort['SalesLastDay'] = df3_sort.groupby('Store')['Sales'].shift(1)

# Média dos últimos 7 dias
df3_sort['MeanSalesLastSevenDays'] = df3_sort.groupby('Store')['Sales'].shift(1).rolling(7).mean()

# Total de vendas dos últimos 7 dias
df3_sort['TotalSalesLastSevenDays'] = df3_sort.groupby('Store')['Sales'].shift(1).rolling(7).sum()

# Total de Promoções aplicadas nos últimos 7 dias
df3_sort['TotalPromoLastSevenDays'] = df3_sort.groupby('Store')['Promo'].shift(1).rolling(7).sum()

# Total de Feriados Públicos nos últimos 7 dias
df3_sort['TotalPublicHolidaysLastSevenDays'] = df3_sort.groupby('Store')['PublicHoliday'].shift(1).rolling(7).sum()

# Total de promoções 2 ativadas nos últimos 7 dias
df3_sort['TotalPromo2LastSevenDays'] = df3_sort.groupby('Store')['Promo2'].shift(1).rolling(7).sum()

# Total de feriados escolares no últimos 7 dias
df3_sort['TotalSchoolHolidayLastSevenDays'] = df3_sort.groupby('Store')['SchoolHoliday'].shift(1).rolling(7).sum()

# Anos de Competição
df3_sort['YearsCompetition'] = df3_sort['Date'].max().year - df3_sort['CompetitionOpenSinceYear']

Aqui vamos selecionar apenas as variáveis que façam sentido para o nosso problema. 

In [72]:
df3_sort = df3_sort[['Store', 'StoreType', 'Assortment', 'CompetitionDistance', 'YearsCompetition', 'Sales', 'Customers', 'Promo','Promo2', 'PublicHoliday', 'SalesLastDay',
                     'MeanSalesLastSevenDays', 'TotalSalesLastSevenDays', 'TotalPromoLastSevenDays', 'TotalPromo2LastSevenDays', 'TotalSchoolHolidayLastSevenDays', 'TotalPublicHolidaysLastSevenDays']]

# Renomeando colunas
df3_sort = df3_sort.rename(columns={'Sales':'SalesDay', 'Customers': 'CustomersDay', 'PromoDay':'Promo2Day', 'PublicHoliday':'PublicHolidayDay'})

Observe que a nossa variável alvo nossa é a TotalSalesNextSixDays. Sendo assim, nossa variável alvo não pode ser nula. Além disso, de modo que tenhamos o máximo de informação consolidada, vamos eliminar a quantidade as variáveis nulas em relação a soma móveis, pois pelo fato de ser nulo, quer dizer que não há informações disponíveis, logo, não está trazendo o conteúdo que precisamos.

In [73]:
# eliminando os dados desnecessários
df3_sort_clean = df3_sort.dropna()

## 4.0 Previsão das Próximos 6 dias de Faturamento

In [80]:
# Cópia segura dos dados
df4 = df3_sort_clean.drop(columns=['Store'], axis=1).copy()

In [ ]:
# Instanciando o modelo
modelo_preditivo = pkl.load(open('../Modelos/XGBRegressorFinalVersao1.pkl', 'rb'))

# Realizando a Predição
predicao = modelo_preditivo.predict(df4)

## 5.0 Construção da Base Final

In [84]:
df5 = df3_sort_clean.copy()

# Armazenando os dados no Dataset
df5.loc[:,'PredictionSalesNextSexWeek'] = np.round(predicao.tolist(), 2)

## 6.0 Salvando os Dados

In [86]:
df5.to_parquet(path='../DadosPredicao/BasePredita.parquet', engine='fastparquet', index=False)